# Librerías

In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

# Instancia

In [2]:
PATH = '../voronoi/2K'
VERTEX_PATH = PATH + '/vertices.txt'
RIDGES_PATH = PATH + '/ridges.txt'
ORIS_PATH = PATH + '/oris.txt'
SE_PATH = PATH + '/SE/uniform.txt'
N_GRAINS = 10 #>=3
SEED = 123
DOMAIN_BOUND = 1.0
eps = 0.2


In [3]:
!python ../voronoi/voronoi.py $N_GRAINS $SEED $VERTEX_PATH $RIDGES_PATH $ORIS_PATH $SE_PATH

Vertices: 20
Ridges: 30
Orientations: 10
Using uniform distribution for SE
SE uniform: 10


In [4]:
ridges = np.loadtxt(RIDGES_PATH, dtype = int)
vertices = np.loadtxt(VERTEX_PATH)
oris = np.loadtxt(ORIS_PATH)
SE = np.loadtxt(SE_PATH)

# Funciones

In [5]:
def wrap_dist(dest, orig, DOMAIN_BOUND):
    delta = dest - orig
    if delta[0] > DOMAIN_BOUND*0.5:
        delta[0] = delta[0] - DOMAIN_BOUND
    elif delta[0] < -0.5*DOMAIN_BOUND:
        delta[0] = DOMAIN_BOUND + delta[0]
    if delta[1] > 0.5*DOMAIN_BOUND:
        delta[1] = delta[1] - DOMAIN_BOUND
    elif delta[1] < -0.5*DOMAIN_BOUND:
        delta[1] = DOMAIN_BOUND + delta[1]
    return delta

In [6]:
def _adjust_points(xpos, ypos, DOMAIN_BOUND):
    rang = len(xpos)
    xret = np.zeros(rang)
    yret = np.zeros(rang)
    xret[:] = xpos
    yret[:] = ypos
    for i in range(1, rang):
        curr = np.array([xret[i], yret[i]])
        prev = np.array([xret[i - 1], yret[i - 1]])
        delta = wrap_dist(curr, prev, DOMAIN_BOUND)
        curr = prev + delta
        xret[i] = curr[0]
        yret[i] = curr[1]
    return xret, yret

In [7]:
def plot_structure(boundarys, vertexs, domain=1.0, lw=1.0, figsize=(10, 7), vertex_plot=False):
    """
        Plot boundaries lines
    """
    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(111)
    coords = [bnd for bnd in boundarys]
    coords2 = [bnd+[domain,0] for bnd in boundarys if np.any(bnd[:,0] < 0)]
    coords3 = [bnd+[-domain,0] for bnd in boundarys if np.any(bnd[:,0] > domain)]
    coords4 = [bnd+[0,domain] for bnd in boundarys if np.any(bnd[:,1] < 0)]
    coords5 = [bnd+[0,-domain] for bnd in boundarys if np.any(bnd[:,1] > domain)]
    coords6 = [bnd+[domain,domain] for bnd in boundarys \
                if np.any(bnd[:,0] < 0) and np.any(bnd[:,1] < 0)]
    coords7 = [bnd+[-domain,-domain] for bnd in boundarys \
                if np.any(bnd[:,0] > domain) and np.any(bnd[:,1] > domain)]
    coords8 = [bnd+[-domain,domain] for bnd in boundarys \
                if np.any(bnd[:,0] > domain) and np.any(bnd[:,1] < 0)]
    coords9 = [bnd+[domain,-domain] for bnd in boundarys \
                if np.any(bnd[:,0] < 0) and np.any(bnd[:,1] > domain)]
    coords = coords+coords2+coords3+coords4+coords5+coords6+coords7+coords8+coords9
    bnd_collection = LineCollection(coords, lw=lw, color='k')
    ax.add_collection(bnd_collection)
    """
        Plot vertexs
    """
    if vertex_plot:
        plt.xlim(0., 1.)
        plt.ylim(0., 1.)
        plt.plot(vertexs[:,0], vertexs[:, 1], "ko", ms=1.0)
    return ax

In [8]:
def get_all_bounds_coords(ridges, vertices):
    boundarys = np.array([[[vertices[bound[0]][0], 
                        vertices[bound[0]][1]],
                       [vertices[bound[1]][0], 
                        vertices[bound[1]][1]]] for bound in ridges])
    return boundarys

In [9]:
def adjust_all_bounds(boundarys, bound_domain):
    boundarys_mod = np.zeros(boundarys.shape)
    for index, bound in enumerate(boundarys):
        ret = _adjust_points(bound[:,0], bound[:,1], bound_domain)
        boundarys_mod[index][:, 0] = ret[0]
        boundarys_mod[index][:, 1] = ret[1]
    return boundarys_mod

In [10]:
def compute_T(v_ini, v_fin, domain):
    dist = wrap_dist(v_fin, v_ini, domain)
    angle = np.arctan2(dist[1], dist[0])
    T_m = np.array([np.cos(angle), np.sin(angle)])
    return T_m

In [12]:
"""
output: [[bound01, bound02, bound03], 
        ...
         [boundn1, boundn2, boundn3]]
Cada fila representa el indice del vertice,
por ejemplo [bound01, bound02, bound03] son
los bounds asociados al vertice 0. 
"""

def associate_all_bound_to_vertex(n_vertices, boundarys):
    vertex_association = np.full((n_vertices, 3), -1, dtype=int)
    for index, (v1, v2) in enumerate(boundarys):
        for i in np.arange(3):
            if vertex_association[v1][i] == -1:
                vertex_association[v1][i] = index
                break
        for i in np.arange(3):
            if vertex_association[v2][i] == -1:
                vertex_association[v2][i] = index
                break
    return vertex_association

In [13]:
def vertex_set_boundaries_clockwise(vertex_association, ridges, vertices, domain):
    l = vertex_association.shape[0]
    for vertex in np.arange(l):
        angle = np.zeros(3)
        for i in np.arange(3):
            ini_v, end_v = ridges[vertex_association[vertex][i]]
            current = vertices[vertex]
            delta = wrap_dist(vertices[end_v], current, domain) if ini_v == vertex else wrap_dist(vertices[ini_v], current, domain)
            angle[i] = np.arctan2(delta[1], delta[0])
        for i in np.arange(2):
            for p in np.arange(1, 3-i):
                if angle[p-1] < angle[p]:
                    angle[p-1], angle[p] = angle[p], angle[p-1]
                    vertex_association[vertex][p-1], vertex_association[vertex][p] = vertex_association[vertex][p], vertex_association[vertex][p-1]

In [14]:
def generate_grains(n_vertices, vertex_association, ridges):
    lengrains = 0
    grains = dict()
    considerated = np.full(3*n_vertices, False)
    for j in np.arange(n_vertices):
        start_v = j
        for g in np.arange(3):
            if not considerated[j*3 + g]:
                grain_vertex_list = list()
                current_v = start_v
                current_side = g
                while True:
                    j_idx = current_v
                    if considerated[j_idx * 3 + current_side]:
                        print("Error.")
                        input("Enter:no se pudo generar la estructura de granos.")
                    considerated[j_idx * 3 + current_side] = True
                    grain_vertex_list.append(current_v)
                    bnd = vertex_association[current_v][current_side]
                    current_v = ridges[bnd][1] if ridges[bnd][0] == current_v else ridges[bnd][0]
                    i = 0
                    while True:
                        if vertex_association[current_v][i] == bnd:
                            break
                        i+=1
                    current_side = (i+1) % 3
                    if current_v == start_v:
                        break
                grains[lengrains] = np.array(grain_vertex_list)
                lengrains += 1
    if lengrains != N_GRAINS:
        print("No calzan los granos.")
    return grains

In [15]:
def bound_energy_func(d_alpha, eps):
    return 1. + 0.5*eps*(1. - np.power(np.cos(4.*d_alpha), 3.))

In [16]:
def associate_grains_to_vertex(grains, n_grains, n_vertices):
    grains_of_vertex = np.full((n_vertices, 3), -1, dtype=int)
    for i in np.arange(n_grains):
        for j in np.arange(grains[i].shape[0]):
            actual_vertex = grains[i][j]
            for k in np.arange(3):
                if grains_of_vertex[actual_vertex][k] == -1:
                    grains_of_vertex[actual_vertex][k] = i
                    break
    return grains_of_vertex

In [17]:
def compute_bounds_energy(ridges, grains_of_vertex, oris, eps):
    all_bounds_energy = np.zeros(ridges.shape[0])
    for bound_index, (v_ini, v_end) in enumerate(ridges):
        k = 0
        grains_id = [0,0]
        for i in np.arange(3):
            actual_grain = grains_of_vertex[v_ini][i]
            if actual_grain in grains_of_vertex[v_end]:
                grains_id[k] = actual_grain
                k += 1
                if k == 2:
                    break
        d_alpha = oris[grains_id[0]] - oris[grains_id[1]]
        all_bounds_energy[bound_index] = bound_energy_func(d_alpha, eps)       
    return all_bounds_energy

In [84]:
def get_neights_and_energys(n_vertices, ridges, bounds_energy):
    neights = np.zeros((n_vertices, 3), dtype=int)
    energys = np.zeros((n_vertices, 3))
    for vertex in np.arange(n_vertices):
        indx_neight, column = np.where(ridges == vertex)
        # map all 0's to 1 and all 1's to 0 using bitwise negation
        column = ~column + 2
        neights[vertex] = ridges[indx_neight, column]
        # search for all bounds_energys by multi indexing
        energys[vertex] = bounds_energy[indx_neight]
    return neights, energys

# Implementación

In [18]:
# Obtener todos los bounds ajustados con condiciones de borde periodicas
# boundarys_mod tiene la siguiente estructura:
"""
[[[inicio_vertex_x_1 inicio_vertex_y_1],
  [fin_vertex_x_1 fin_vertex_y_1]],
  [[inicio_vertex_x_2 inicio_vertex_y_2],
  [fin_vertex_x_2 fin_vertex_y_2]...]
  Donde 1 representa el boundary 1 (el que esta en la posicion 0 del array ridges).
  La estructura de datos contiene todos los boundarys.
"""
boundarys = get_all_bounds_coords(ridges, vertices)
boundarys_mod = adjust_all_bounds(boundarys, DOMAIN_BOUND)

In [51]:
# A cada vertice se le asocian 3 boundarys en sentido horario.
"""
Estructura de vertex_association:
[[bound01, bound02, bound03], 
        ...
[boundn1, boundn2, boundn3]]
Cada fila representa el indice del vertice,
por ejemplo [bound01, bound02, bound03] son
los bounds asociados al vertice 0.
"""
N_VERTICES = vertices.shape[0]
vertex_association = associate_all_bound_to_vertex(N_VERTICES, ridges)
vertex_set_boundaries_clockwise(vertex_association, ridges, vertices, 1.0)

In [20]:
# A cada grano se le asocian vertices:
"""
Estructura de grains:
{
    grano_0:[vertice_1_0, vertice_2_0, ...],
    grano_1:[vertice_1_1, vertice_2_1, ...],
    ...
    grano_n:[vertice_1_n, vertice_2_n, ...]
}
Donde grano_i es un numero que representa a un grano, que coincide con el indice de el array 'oris', 
y vertice_algo_i representan los indices de los vertices asociados al grano i en el array 'vertices'.
"""
N_VERTICES = vertices.shape[0]
grains = generate_grains(N_VERTICES, vertex_association, ridges)

In [21]:
# A cada vertices se le asocian granos:
"""
Estructura de grains_of_vertex:
[[grano_0_0, grano_1_0, grano_2_0],
 [grano_1_1, grano_2_1, grano_3_1],
    ...
 [grano_1_n, grano_2_n, grano_3_n]]
Donde cada fila representa un indice de vertice, por ejemplo
[grano_0_0, grano_1_0, grano_2_0] son los 3 granos asociados a vertices[0]
"""
grains_of_vertex = associate_grains_to_vertex(grains, N_GRAINS, N_VERTICES)

In [22]:
# Calcular las energias de cada boundary:
"""
Estructura de bound_energy:
[energy_bound_0, energy_bound_1, ..., energy_bound_n]

"""
bounds_energy = compute_bounds_energy(ridges, grains_of_vertex, oris, eps)

In [ ]:
def grain_motion(vertex_association, ridges, bounds_energy, n_vertices, vertices):
    
    for vertex in np.arange(n_vertices):
        #ahora tengo un vertice en vertex
        neights_energys = get_neights_and_energys(vertex, vertex_association, 
                                                    ridges, bounds_energy)
        sum = np.zeros(2)
        for ngh, eng in neights_energys.items():
            T = compute_T(vertices[i, vertex], vertices[i, ngh], domain)
            sum += T*eng
    return None

In [30]:
def eulerMethod(t0, T, N, y0, f):
    t = np.linspace(t0, T, N+1)
    h = (T-t0)/N
    y = np.zeros(N+1)
    y[0] = y0
    for i in np.arange(N):
        y[i+1] = y[i]+f(t[i],y[i])*h
    return t, y

In [38]:
def euler_solve(t0, T, N, vertices, vertex_association, ridges, bounds_energy, domain=1.0):
    t = np.linspace(t0, T, N+1)
    h = (T-t0)/N
    vertices_solve = np.zeros((N+1, vertices.shape[0], 2))
    vertices_solve[0, : ,:] = vertices
    n_vertices = vertices.shape[0]
    for i in np.arange(N):
        for vertex in np.arange(n_vertices):
            #ahora tengo un vertice en vertex
            neights_energys = get_neights_and_energys(vertex, vertex_association, 
                                                      ridges, bounds_energy)
            print(neights_energys)
            sum = np.zeros(2)
            for ngh, eng in neights_energys.items():
                T = compute_T(vertices_solve[i, vertex], vertices_solve[i, ngh], domain)
                sum += T*eng
            vertices_solve[i+1, vertex] = vertices_solve[i, vertex]+sum*h
    return t, vertices_solve

In [39]:
t, solve = euler_solve(0, 1, 2, vertices, vertex_association, ridges, bounds_energy, DOMAIN_BOUND)

{1: 1.072239326682146, 2: 1.0324024522498303, 16: 1.0180446634783251}
{6: 1.0806172898446662, 0: 1.072239326682146, 13: 1.0013049389761848}
{0: 1.0324024522498303, 12: 1.0332964213197078, 3: 1.0000097146212417}
{2: 1.0000097146212417, 10: 1.1000019986828056, 5: 1.1000043547539269}
{10: 1.0035680306209556, 9: 1.0362331184544105, 11: 1.0197935941211678}
{8: 1.100348292988765, 3: 1.1000043547539269, 18: 1.0020363094343643}
{15: 1.0999937217462776, 17: 1.149656327747597, 1: 1.0806172898446662}
{18: 1.113259087106958, 11: 1.096331059583186, 12: 1.0735514271052062}
{16: 1.0998278862987496, 5: 1.100348292988765, 9: 1.0106511561590386}
{4: 1.0362331184544105, 8: 1.0106511561590386, 17: 1.0098781741675544}
{3: 1.1000019986828056, 19: 1.0997911353217806, 4: 1.0035680306209556}
{4: 1.0197935941211678, 15: 1.1000064214006298, 7: 1.096331059583186}
{7: 1.0735514271052062, 14: 1.0185708247994671, 2: 1.0332964213197078}
{15: 1.0009519126625273, 1: 1.0013049389761848, 14: 1.000028147700835}
{12: 1.018

# Plot section: No es ejecutable desde VS CODE.

In [35]:
%matplotlib notebook
from ipywidgets import *
figsize = (4, 4)
LINE_WIDTH = 0.1
VERTEX_PLOT = False


def update(time=0):
    plt.close()
    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(111)
    boundarys = get_all_bounds_coords(ridges, solve[time])
    boundarys_mod = adjust_all_bounds(boundarys, DOMAIN_BOUND)
    coords = [bnd for bnd in boundarys_mod]
    coords2 = [bnd+[DOMAIN_BOUND,0] for bnd in boundarys_mod if np.any(bnd[:,0] < 0)]
    coords3 = [bnd+[-DOMAIN_BOUND,0] for bnd in boundarys_mod if np.any(bnd[:,0] > DOMAIN_BOUND)]
    coords4 = [bnd+[0,DOMAIN_BOUND] for bnd in boundarys_mod if np.any(bnd[:,1] < 0)]
    coords5 = [bnd+[0,-DOMAIN_BOUND] for bnd in boundarys_mod if np.any(bnd[:,1] > DOMAIN_BOUND)]
    coords6 = [bnd+[DOMAIN_BOUND,DOMAIN_BOUND] for bnd in boundarys_mod \
                if np.any(bnd[:,0] < 0) and np.any(bnd[:,1] < 0)]
    coords7 = [bnd+[-DOMAIN_BOUND,-DOMAIN_BOUND] for bnd in boundarys_mod \
                if np.any(bnd[:,0] > DOMAIN_BOUND) and np.any(bnd[:,1] > DOMAIN_BOUND)]
    coords8 = [bnd+[-DOMAIN_BOUND,DOMAIN_BOUND] for bnd in boundarys_mod \
                if np.any(bnd[:,0] > DOMAIN_BOUND) and np.any(bnd[:,1] < 0)]
    coords9 = [bnd+[DOMAIN_BOUND,-DOMAIN_BOUND] for bnd in boundarys_mod \
                if np.any(bnd[:,0] < 0) and np.any(bnd[:,1] > DOMAIN_BOUND)]
    coords = coords+coords2+coords3+coords4+coords5+coords6+coords7+coords8+coords9
    bnd_collection = LineCollection(coords, lw=LINE_WIDTH, color='k')
    ax.add_collection(bnd_collection)
    ax.set_ylim(ymin=0.0, ymax=1.0)
    ax.set_xlim(xmin=0.0, xmax=1.0)
interact(update, time=(0, t.shape[0]));

interactive(children=(IntSlider(value=0, description='time', max=11), Output()), _dom_classes=('widget-interac…

In [36]:
# Acá es para  generar un gif
import os
import glob
import imageio

figsize = (4, 4)
LINE_WIDTH = 0.1
VERTEX_PLOT = False
try:
    os.remove('./mygif.gif')
except:
    pass

for time in np.arange(t.shape[0]):
    plt.close()
    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(111)
    boundarys = get_all_bounds_coords(ridges, solve[time])
    boundarys_mod = adjust_all_bounds(boundarys, DOMAIN_BOUND)
    coords = [bnd for bnd in boundarys_mod]
    coords2 = [bnd+[DOMAIN_BOUND,0] for bnd in boundarys_mod if np.any(bnd[:,0] < 0)]
    coords3 = [bnd+[-DOMAIN_BOUND,0] for bnd in boundarys_mod if np.any(bnd[:,0] > DOMAIN_BOUND)]
    coords4 = [bnd+[0,DOMAIN_BOUND] for bnd in boundarys_mod if np.any(bnd[:,1] < 0)]
    coords5 = [bnd+[0,-DOMAIN_BOUND] for bnd in boundarys_mod if np.any(bnd[:,1] > DOMAIN_BOUND)]
    coords6 = [bnd+[DOMAIN_BOUND,DOMAIN_BOUND] for bnd in boundarys_mod \
                if np.any(bnd[:,0] < 0) and np.any(bnd[:,1] < 0)]
    coords7 = [bnd+[-DOMAIN_BOUND,-DOMAIN_BOUND] for bnd in boundarys_mod \
                if np.any(bnd[:,0] > DOMAIN_BOUND) and np.any(bnd[:,1] > DOMAIN_BOUND)]
    coords8 = [bnd+[-DOMAIN_BOUND,DOMAIN_BOUND] for bnd in boundarys_mod \
                if np.any(bnd[:,0] > DOMAIN_BOUND) and np.any(bnd[:,1] < 0)]
    coords9 = [bnd+[DOMAIN_BOUND,-DOMAIN_BOUND] for bnd in boundarys_mod \
                if np.any(bnd[:,0] < 0) and np.any(bnd[:,1] > DOMAIN_BOUND)]
    coords = coords+coords2+coords3+coords4+coords5+coords6+coords7+coords8+coords9
    bnd_collection = LineCollection(coords, lw=LINE_WIDTH, color='k')
    ax.add_collection(bnd_collection)
    ax.set_ylim(ymin=0.0, ymax=1.0)
    ax.set_xlim(xmin=0.0, xmax=1.0)
    plt.savefig('../images/{}.png'.format(time))

with imageio.get_writer('mygif.gif', mode='I') as writer:
    for filename in ['../images/{}.png'.format(i) for i in range(0, t.shape[0])]:
        image = imageio.imread(filename)
        writer.append_data(image)

files = glob.glob('../images/*')
for f in files:
    print(f)
    os.remove(f)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

C:\Users\Cristóbal Abarca\AppData\Local\Temp\ipykernel_16728\2030926524.py:42: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


../images\0.png
../images\1.png
../images\10.png
../images\2.png
../images\3.png
../images\4.png
../images\5.png
../images\6.png
../images\7.png
../images\8.png
../images\9.png
